In [6]:
##https://github.com/microsoft/SynapseML/blob/master/docs/Use%20with%20MLFlow/Overview.md
##https://github.com/microsoft/SynapseML/blob/master/docs/Explore%20Algorithms/Hyperparameter%20Tuning/HyperOpt.ipynb

## 3. Model scoring

In [7]:
# Importing pyspark dependencies
from pyspark.sql.types import IntegerType,BooleanType,DateType,NumericType,TimestampType
from pyspark.ml.feature import Binarizer, Bucketizer, QuantileDiscretizer
from com.microsoft.spark.sqlanalytics.Constants import Constants
import com.microsoft.spark.sqlanalytics
from pyspark.sql.functions import col
import pyspark.sql.functions as F

# Importing mlflow libraries
from mlflow.models import infer_signature, set_signature
from mlflow.models.model import get_model_info
import mlflow


# Importing general libraries
from sklearn.metrics import accuracy_score, f1_score, precision_score, confusion_matrix, recall_score, roc_auc_score, classification_report
import pandas as pd

#AML workspace authentication using linked service
from notebookutils.mssparkutils import azureML
linked_service_name = "AzureMLService1"
ws = azureML.getWorkspace(linked_service_name)
mlflow.set_tracking_uri(ws.get_mlflow_tracking_uri().replace("v2.0","v1.0"))

## Importing SynapseML
from synapse.ml.featurize import Featurize
from synapse.ml.lightgbm import *
from synapse.ml.train import ComputeModelStatistics

### 3.1 Importing data

In [8]:
# Read from existing internal table
df = (spark.read.synapsesql("synapseazuremldedicates.dbo.class_transformed_taxi_data")).drop('cost')

# Show contents of the dataframe
display(df.head(5))

### 3.2  Testing model from run folder

In [9]:
### --------------------------------------------------------------------------- ###
model_name = 'featurizer'
# Search registered models for the 
for model in mlflow.search_registered_models():
    if model.name == model_name:
        latest_version_run_id = model.latest_versions[0].run_id
        model_found = True
        break

# load model from Azure Machine Learning
run_id = latest_version_run_id
artifact_path = "model"
model_uri = f"runs:/{run_id}/{artifact_path}"
featurizer_model = mlflow.spark.load_model(model_uri)

### --------------------------------------------------------------------------- ###

model_name = 'classification_demo'
# Search registered models for the 
for model in mlflow.search_registered_models():
    if model.name == model_name:
        latest_version_run_id = model.latest_versions[0].run_id
        model_found = True
        break

# load model from Azure Machine Learning
run_id = latest_version_run_id
artifact_path = "lightgbm_model"
model_uri = f"runs:/{run_id}/{artifact_path}"
model = mlflow.pyfunc.load_model(model_uri)

### --------------------------------------------------------------------------- ###

df_trans = featurizer_model.transform(df)
pandas_df = df_trans.toPandas()

# testing model
test_predictions = model.predict(pandas_df.head())
print(test_predictions)

### 3.3 Testing model from model registry using pyfunc

In [10]:
model = mlflow.pyfunc.load_model(model_uri=f"models:/classification_demo/latest")

test_predictions = model.predict(pandas_df)
print(test_predictions)

### 3.4 Testing model from model registry using spark

In [11]:
spark_model = mlflow.spark.load_model(model_uri=f"models:/classification_demo/latest")
display(spark_model.transform(df_trans))